# Deep Reinforcement Learning

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, a platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [1]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [2]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemnt. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [4]:
env_name = 'CartPole-v1'

# Create an instance of the environment
env = gym.make("CartPole-v1", render_mode="rgb_array")

env.reset()

frames = []

for _ in range(500):
    action = np.random.randint(0, env.action_space.n)

    obs, reward, done, _ = env.step(action)

    # render this frame and add to the list of frames

    frames.append(env.render()[0])

    if done:
        env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym

In [36]:
print(env.reset())
print(env.step(1))
print(env.observation_space.shape)


[ 0.0444686  -0.04569073  0.02905543 -0.04964752]
(array([ 0.04355479,  0.1490028 ,  0.02806247, -0.33302346], dtype=float32), 1.0, False, {})
(4,)


In [5]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.

You need to implement the convolutional neural network.


In [7]:
class ActorCritic(nn.Module):
    def __init__(self, num_input, hidden_size, num_actions):
        super(ActorCritic, self).__init__()
        # TODO: define your model here
        self.fc1 = nn.Linear(num_input, hidden_size)
        self.actor = nn.Linear(hidden_size, num_actions)
        self.critic = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        action_probs = torch.softmax(self.actor(x), dim=-1)
        value = self.critic(x)
        return action_probs, value


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [55]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = int(num_episodes)
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


        # self.input_dim = self.env.observation_space.shape[0]
        # self.num_actions = self.env.action_space.n

        # define your actor critic network, its optimzer and the loss function
        self.policy_net = ActorCritic(self.env.observation_space.shape[0], hidden_size, self.env.action_space.n).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        self.critic_loss = nn.MSELoss()

    def choose_action(self, state):
        # TODO
        # choose an action by sampling from the policy distribution
        state = torch.tensor(state, dtype=torch.float).to(self.device)
        action_probs, _ = self.policy_net(state)
        action = torch.multinomial(action_probs, 1).item()
        return action, torch.log(action_probs[action])
        # return action

    def compute_returns(self, rewards):
        # TODO
        # calculate the discounted rewards used in the training process
        new_rewards = []
        R = 0
        for r in reversed(rewards):
            R = r + self.gamma * R
            new_rewards.insert(0, R)
        return new_rewards

    def train(self):
        episode_rewards = np.array([])
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                # TODO
                # Implement the training loop
                state = self.env.reset()
                episode_reward = 0
                log_probs = []
                values = []
                rewards = []

                for step in range(self.max_steps):
                    # TODO
                    # first you need to gather log probabilities, state values and rewards from a trajectory
                    # update the 'episode_reward'
                    action, log_prob = self.choose_action(state)
                    next_state, reward, done, _ = self.env.step(action)

                    _, value = self.policy_net(torch.tensor(state, dtype=torch.float).to(self.device))

                    log_probs.append(log_prob)
                    values.append(value)
                    rewards.append(reward)

                    episode_reward += reward
                    state = next_state

                    if done:
                        break



                episode_rewards = np.append(episode_rewards, episode_reward)

                # if not rewards:
                #     continue

                # calculate the discounted rewards
                returns = self.compute_returns(rewards)
                returns = torch.tensor(returns).to(self.device)


                log_probs = torch.stack(log_probs)
                values = torch.stack(values).squeeze()

                # print(values.size())

                # calculate advantage
                advantage = returns - values

                # compute actor and critic losses
                actor_loss = -(log_probs * advantage.detach()).mean()
                critic_loss = self.critic_loss(values, returns)
                loss = actor_loss + critic_loss

                # TODO
                # update the model after performing the backward pass
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                pbar.set_description(f"Episode {episode}, Reward: {np.mean(episode_reward)}")

        self.env.close()
        return episode_rewards

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Define the model and set hyperparameters.

In [56]:
env = gym.make('CartPole-v1')
num_episodes = 2000
max_steps = 500
lr = 1e-3
hidden_size = 256

a2c_model = A2CAgent(env, num_episodes=num_episodes)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Train the model.

In [57]:
rewards = a2c_model.train()

  0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Episode 1999, Reward: 420.0: 100%|██████████| 2000/2000 [16:08<00:00,  2.06it/s]


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [58]:
env = gym.make(env_name, render_mode='rgb_array')

model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# complete the loop and gather the episode reward of each episode as well the frames of the first episode
frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0

    # TODO
    # Implement the evaluation loop
    env.reset()
    for step in range(500):
      action_probs, _ = model.policy_net.forward(torch.tensor(state).to(device))
      action = torch.argmax(action_probs).cpu().numpy()
      # print(action)
      new_state, reward, done , _ = env.step(action)
      # print(new_state)
      episode_reward += reward
      state = new_state
      if step == 0:
        frames.append(env.render()[0])
      if done:
        break

    episode_rewards.append(episode_reward)
    print(f"Episode Reward: {episode_reward}")
env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
imageio.mimsave('./test.mp4', frames, fps=25)

Episode Reward: 452.0
Episode Reward: 342.0
Episode Reward: 453.0
Episode Reward: 421.0
Episode Reward: 370.0
Episode Reward: 305.0
Episode Reward: 500.0
Episode Reward: 494.0
Episode Reward: 331.0


Episode Reward: 457.0
Average Reward over 10 episodes: 412.5


In [59]:
show_video('./test.mp4')